In [1]:
from dotenv import load_dotenv

load_dotenv()
from falkordb_gemini_kg import (
    Ontology,
    Entity,
    Relation,
    Attribute,
    AttributeType,
    KnowledgeGraph,
    KnowledgeGraphModelConfig,
)
from falkordb_gemini_kg.orchestrator import Orchestrator
from falkordb_gemini_kg.agents.kg_agent import KGAgent
from falkordb_gemini_kg.models.openai import OpenAiGenerativeModel

from json import loads
from falkordb import FalkorDB
import os
import vertexai
import logging

logging.basicConfig(level=logging.DEBUG)

vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("REGION"))

restaurants_ontology = Ontology()
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)


attractions_ontology = Ontology()
attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)


model = OpenAiGenerativeModel("gpt-4o")
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)


def import_data():
    cities = loads(open("data/cities.json").read())
    restaurants = loads(open("data/restaurants.json").read())
    attractions = loads(open("data/attractions.json").read())

    for city in cities:
        restaurants_kg.add_node(
            "City",
            {
                "name": city["name"],
                "weather": city["weather"],
                "population": city["population"],
            },
        )
        restaurants_kg.add_node("Country", {"name": city["country"]})
        restaurants_kg.add_edge(
            "IN_COUNTRY",
            "City",
            "Country",
            {"name": city["name"]},
            {"name": city["country"]},
        )

        attractions_kg.add_node(
            "City",
            {
                "name": city["name"],
                "weather": city["weather"],
                "population": city["population"],
            },
        )
        attractions_kg.add_node("Country", {"name": city["country"]})
        attractions_kg.add_edge(
            "IN_COUNTRY",
            "City",
            "Country",
            {"name": city["name"]},
            {"name": city["country"]},
        )

    for restaurant in restaurants:
        restaurants_kg.add_node(
            "Restaurant",
            {
                "name": restaurant["name"],
                "description": restaurant["description"],
                "rating": restaurant["rating"],
                "food_type": restaurant["food_type"],
            },
        )
        restaurants_kg.add_edge(
            "IN_CITY",
            "Restaurant",
            "City",
            {"name": restaurant["name"]},
            {"name": restaurant["city"]},
        )

    for attraction in attractions:
        attractions_kg.add_node(
            "Attraction",
            {
                "name": attraction["name"],
                "description": attraction["description"],
                "type": attraction["type"],
            },
        )
        attractions_kg.add_edge(
            "IN_CITY",
            "Attraction",
            "City",
            {"name": attraction["name"]},
            {"name": attraction["city"]},
        )

In [2]:
import_data()

In [4]:
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="""
I'm a restaurant agent, specialized in finding the best restaurants for you. 
""",
)

attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="""
I'm an attractions agent, specialized in finding the best attractions for you. 
""",
)

orchestrator = Orchestrator(model)

orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

runner = orchestrator.ask("Write me a 3 day itinerary for a trip to Italy")

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/aviavni/repos/falkordb-gemini-kg/venv/lib/python3.12/site-packages/certifi/cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '\nYou are an orchestrator agent that manages the flow of information between different agent, in order to provide a complete and accurate answer to the user\'s question.\nYou will receive a question that requires information from different agents to answer.\nFor that to happen in the most efficient way, you will create an execution plan where every step will be performed by other agent.\nBe sure to ask the user for more information to answer the question in the most accurate way, unless explicitly told otherwise.\nAfter every step, you will decide what to do next based on the information you have.\nOnce all the step

In [6]:
print(runner.output)

Thank you for your patience. Based on the information gathered, here is a detailed 3-day itinerary for your trip to Italy, encompassing both top attractions and the local dining experience.

### Day 1: Rome
Rome, a city steeped in history and culture, will be your first stop.

**Morning:**
1. **Colosseum**: Start your day by visiting the ancient Colosseum. This iconic amphitheater is known for its gladiatorial contests and historic spectacles.
2. **Roman Forum**: Nearby, explore the Roman Forum, the hub of ancient Roman public life.

**Lunch:**
- **Local Suggestion**: Enjoy a traditional Roman meal at a local trattoria. Look for places that offer classic dishes like Carbonara or Cacio e Pepe.

**Afternoon:**
1. **Vatican Museums**: Head towards Vatican City to visit the Vatican Museums, home to centuries of art collected by Popes, including Michelangelo's Sistine Chapel ceiling.
2. **St. Peter's Basilica**: Take a moment to visit St. Peter's Basilica, one of the largest and most signif